## Setup

In [17]:
!pip install --user -U nltk

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random
import nltk
from nltk.tokenize import sent_tokenize


## Input data directory
data_dir = "cureus2"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [2]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
#loader = PyPDFLoader("./data_input/1-s2.0-S1347436721000355-main.pdf")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[5].page_content)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.47s/it]

Number of chunks =  39
These alerts are automatically presented in the prescribing module of the electronic health record system, as buttons highlighted in white, yellow and red, respec-tively. Therefore, the prescribing physicians were exposed to, and had the chance to react to, the same alerts as did the assessors. We also entered each patient’s current medica-tion list into the open-access interface of J anusmed (Janu- ary 2020) [17] to retrieve the specific recommendations provided to manage the alerts [7 ]. Additional medically justified actions prior to the next regular visit were recorded if they were related to the interaction alerts, as determined retrospectively by the assessors in consensus. For instance, this could include the switch or withdrawal of a drug, the ordering of a labo-ratory test, the retrieval of more information about the patient, or arranging an extra visit. Dosing was considered in the assessments. Patients’ characteristics included age, sex, residence, and

## Create a dataframe of all the chunks

In [3]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df=df[1:20]
df.head()
df.to_csv("chunksnotprocessed.csv", index=False)



(39, 3)


In [21]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_ner_bionlp13cg_md-0.5.3.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.7 MB/s eta 0:00:0000:0100:03
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 5.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858.7/858.7 kB 2.6 MB/s eta 0:00:0000:0100:01
  Created wheel for en-ner-bionlp13cg-md: filename=en_ner_bionlp13cg_md-0.5.3-py3-none-any.whl size=119816876 sha256=a0c1e864ee9f4981f1bd50dc0307369a8119e70c4db551bfa8a5e1c5690cc54b
  Stored in directory: /Users/lauraolivaresbenitez/Library/Caches/pip/wheels/fd/2b/53/6d36d4c44cded3a649617560bfaaa3a9bdb3f7e3bab37e553a
Successfully built en-ner-bionlp13cg-md
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.2
    Uninstalling thinc-8.2.2:
      Successfully uninstalled thinc-8.2.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.2
    Uninstalling spacy-3.7.2:
      Successfully uninstalled spacy-3.7.2
ERROR: pip's dependency resolver does

In [4]:
import spacy
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lauraolivaresbenitez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lauraolivaresbenitez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lauraolivaresbenitez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
# Cargar el modelo de Spacy
nlp = spacy.load("en_ner_bionlp13cg_md")

# Función para limpiar el texto
def clean_chunk(chunk):
    # Eliminar números y signos de puntuación, excepto guiones en palabras compuestas
    clean_text = re.sub(r'\d+', '', chunk)  # Eliminar números
    #clean_text = re.sub(r'(?<=\w)-(?! )|(?<! )-(?=\w)', '', clean_text)  # Eliminar guiones que no están precedidos ni seguidos por espacios
    #clean_text = re.sub(r'[^\w\s-]', '', clean_text)  # Eliminar signos de puntuación, conservando el guión
    
    # Convertir a minúsculas
    clean_text = clean_text.lower()
    
    # Tokenizar el texto
    tokens = word_tokenize(clean_text)
    
    # Eliminar stopwords y palabras de longitud 1
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
    
    # Lematizar las palabras
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Unir tokens limpios en un solo texto
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

for index, row in df.iterrows():
    # Limpiar el texto y luego pasarlo a Spacy
    cleaned_text = clean_text(row['text'])
    doc = nlp(cleaned_text)

# Muestra las primeras filas del DataFrame, con las entidades encontradas
print(df.head())
df.to_csv("chunksprocessed.csv", index=False)

                                                text  \
1  These actions most often involved a switch to ...   
2  Several web-based electronic databases have be...   
3  There are various reasons for overriding an al...   
4  In that study, the drug treatment of each pati...   
5  These alerts are automatically presented in th...   

                                         source  \
1  data_input/cureus2/228_2022_Article_3292.txt   
2  data_input/cureus2/228_2022_Article_3292.txt   
3  data_input/cureus2/228_2022_Article_3292.txt   
4  data_input/cureus2/228_2022_Article_3292.txt   
5  data_input/cureus2/228_2022_Article_3292.txt   

                           chunk_id  
1  6bfa6c63add8416cae0c0b3a8398fbb5  
2  7fba24b9600d420bb6e2389d52ba91d3  
3  97d2c1ede0c248ef918761adc482d6d9  
4  a872f33db19c4276b6fcea139ce95658  
5  94cc8fe1b51444de9e589ffca030afde  


In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_chunk(chunk):
    # Eliminar números y signos de puntuación, excepto guiones en palabras compuestas
    clean_text = re.sub(r'\d+', '', chunk)  # Eliminar números
    #clean_text = re.sub(r'(?<=\w)-(?! )|(?<! )-(?=\w)', '', clean_text)  # Eliminar guiones que no están precedidos ni seguidos por espacios
    #clean_text = re.sub(r'[^\w\s-]', '', clean_text)  # Eliminar signos de puntuación, conservando el guión
    
    # Convertir a minúsculas
    clean_text = clean_text.lower()
    
    # Tokenizar el texto
    tokens = word_tokenize(clean_text)
    
    # Eliminar stopwords y palabras de longitud 1
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
    
    # Lematizar las palabras
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Unir tokens limpios en un solo texto
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Texto de ejemplo
texto = "=4) Antacids–ferrous sulphate 3 Decreased absorption of ferrous sulphate Separate the intake (n =1) Verify separated intake at next visit (n =2)1120 European Journal of Clinical Pharmacology (2022) 78:1115–11261 3 Table 4 (continued) Interaction pair Janusmed aler taPhysician assessmentb Diclofenac–metoprolol 3 NSAID can in some patients decrease the antihypertensive effect of beta-adrenergic receptor antagonists(n = 0) Diclofenac PRN (n = 2) Episodic use (n =1) Simvastatin–warfarin 3 Increased effect of warfarin may occur; increased risk of bleeding(n = 0) Stable warfarin dose, INR regularly monitored (n =3) Amiloride–diclofenac 2 NSAID may impair the diuretic and antihypertensive effect; acute kidney failure may occur; the combination increases the risk of stomach ulcersStop diclofenac (n =1) Diclofenac PRN (n =1) Diclofenac–SSRI 2 Markedly increased risk of GI bleeding Stop diclofenac (n =1) Already on gastroprotection with a PPI (n =1) Ferrous sulphate–levothyroxine 2 Decreased effect of levothyroxine Separate the intake (n =2) (n =0) Antacids–levothyroxine 1 Decreased absorption of levothyroxine Separate the intake (n =1) (n =0) Chlorzoxazone–simvastatin 1 Rhabdomyolysis and cholestasis may occur Stop chlorzoxazone (n =1) (n =0) Clopidogrel–pioglitazone 1 Increased exposure to pioglitazone, with increased risk of hypoglycaemia and other dose-related adverse effectsHalve the dose of pioglitazone (n =1)(n ="

# Limpiar el texto
texto_limpio = clean_chunk(texto)
print(texto)
print(texto_limpio)



=4) Antacids–ferrous sulphate 3 Decreased absorption of ferrous sulphate Separate the intake (n =1) Verify separated intake at next visit (n =2)1120 European Journal of Clinical Pharmacology (2022) 78:1115–11261 3 Table 4 (continued) Interaction pair Janusmed aler taPhysician assessmentb Diclofenac–metoprolol 3 NSAID can in some patients decrease the antihypertensive effect of beta-adrenergic receptor antagonists(n = 0) Diclofenac PRN (n = 2) Episodic use (n =1) Simvastatin–warfarin 3 Increased effect of warfarin may occur; increased risk of bleeding(n = 0) Stable warfarin dose, INR regularly monitored (n =3) Amiloride–diclofenac 2 NSAID may impair the diuretic and antihypertensive effect; acute kidney failure may occur; the combination increases the risk of stomach ulcersStop diclofenac (n =1) Diclofenac PRN (n =1) Diclofenac–SSRI 2 Markedly increased risk of GI bleeding Stop diclofenac (n =1) Already on gastroprotection with a PPI (n =1) Ferrous sulphate–levothyroxine 2 Decreased eff

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lauraolivaresbenitez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lauraolivaresbenitez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lauraolivaresbenitez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Extract Concepts

In [6]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [18]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    
    concepts_list = df2Graph(df, model='zephyr:latest') 
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=True)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=True)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  

print(dfg1.shape)
dfg1.head()


{
  "node_1": "Drug-drug interaction alert",
  "node_2": "Interaction database",
  "edge": "HAS_RELATIONSHIP",
  "properties": {}
},
{
  "node_1": "Medication therapy management",
  "node_2": "Older people",
  "edge": "IS_FOR",
  "properties": {}
},
{
  "node_1": "Polypharmacy",
  "node_2": "Older people",
  "edge": "IS_FOR",
  "properties": {}
},
{
  "node_1": "Primary care",
  "node_2": "Keywords",
  "edge": "IS_A_PART_OF",
  "properties": {}
},
{
  "node_1": "Interaction alerts",
  "node_2": "Drug-drug interaction alert",
  "edge": "IS_A_TYPE_OF",
  "properties": {}
},
{
  "node_1": "Applicable actions",
  "node_2": "Alert system",
  "edge": "IS_A_TYPE_OF",
  "properties": {}
},
{
  "node_1": "Laboratory parameters",
  "node_2": "Laboratory test",
  "edge": "IS_A_TYPE_OF",
  "properties": {}
},
{
  "node_1": "Switch to a less interacting drug",
  "node_2": "Actions suggested by the alert system",
  "edge": "IS_A_TYPE_OF",
  "properties": {}
},
{
  "node_1": "Separate intake",
  "nod

,node_1,node_2,edge,properties,chunk_id,count
0,decision support systems,alert fatigue,HAS_INTERACTION,{},7fba24b9600d420bb6e2389d52ba91d3,4
1,web-based electronic databases,potentially problematic drug interactions,IDENTIFIES,{},7fba24b9600d420bb6e2389d52ba91d3,4
2,systematic review,prevalence of potential drug interactions,IDENTIFIES,{},7fba24b9600d420bb6e2389d52ba91d3,4
3,systematic review,prevalence of clinically manifested drug inter...,IDENTIFIES,{},7fba24b9600d420bb6e2389d52ba91d3,4
4,clinical decision support systems,usefulness during patient consultation,HAS_INTERACTION,{},7fba24b9600d420bb6e2389d52ba91d3,4


In [ ]:
import pandas as pd
from neo4j import GraphDatabase

def create_interaction_relationships(file_path, neo4j_uri, neo4j_user, neo4j_password):
    # Se lee el archivo graph csv con delimitadores |
    df = pd.read_csv(file_path, delimiter='|')
    
    # Conecta con la base de datos Neo4j
    with GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password)) as driver:
        with driver.session() as session:
            # Itera sobre las filas del DataFrame
            for index, row in df.iterrows():
                # Extrae información de la fila
                node1 = row['node_1'].lower().capitalize()
                node2 = row['node_2'].lower().capitalize()
                edge = row['edge']
                properties = row['properties']
                # Parsea las propiedades desde el formato string a un diccionario
                properties_dict = eval(properties) if pd.notna(properties) else {}
                # Construye la consulta Cypher según el tipo de interacción
                if edge == "HAS_INTERACTION":
                    cypher_query = f"""
                        OPTIONAL MATCH (node1:Drug {{Drug_name: "{node1}"}})
                        OPTIONAL MATCH (node2:Drug {{Drug_name: "{node2}"}})
                        WHERE node1 IS NOT NULL AND node2 IS NOT NULL
                        MERGE (node1)-[:HAS_INTERACTION_NLP {{
                            type: "{properties_dict.get("type", "")}",
                            severity: "{str(properties_dict.get("severity", ""))}",
                            gender: "{properties_dict.get("gender", "")}",
                            pregnancy: "{properties_dict.get("pregnancy", "")}",
                            contraindications: "{properties_dict.get("contraindications", "")}",
                            adverse_effects: "{properties_dict.get("adverse_effects", "")}"
                        }}]->(node2)
                        MERGE (node2)-[:HAS_INTERACTION_NLP {{
                            type: "{properties_dict.get("type", "")}",
                            severity: "{str(properties_dict.get("severity", ""))}",
                            gender: "{properties_dict.get("gender", "")}",
                            pregnancy: "{properties_dict.get("pregnancy", "")}",
                            contraindications: "{properties_dict.get("contraindications", "")}",
                            adverse_effects: "{properties_dict.get("adverse_effects", "")}"
                        }}]->(node1);
                    """
                elif edge in ["DECREASED_ABSORPTION", "DECREASED_EFFECT"]:
                    interaction_type = "Decreased Effect"
                    cypher_query = f"""
                        OPTIONAL MATCH (node1:Drug {{Drug_name: "{node1}"}})
                        OPTIONAL MATCH (node2:Drug {{Drug_name: "{node2}"}})
                        WHERE node1 IS NOT NULL AND node2 IS NOT NULL
                        MERGE (node1)-[:HAS_INTERACTION_NLP {{
                            type: "{interaction_type}",
                            severity: "{str(properties_dict.get("severity", ""))}",
                            gender: "{properties_dict.get("gender", "")}",
                            pregnancy: "{properties_dict.get("pregnancy", "")}",
                            contraindications: "{properties_dict.get("contraindications", "")}",
                            adverse_effects: "{properties_dict.get("adverse_effects", "")}"
                        }}]->(node2)
                        MERGE (node2)-[:HAS_INTERACTION_NLP {{
                            type: "{interaction_type}",
                            severity: "{str(properties_dict.get("severity", ""))}",
                            gender: "{properties_dict.get("gender", "")}",
                            pregnancy: "{properties_dict.get("pregnancy", "")}",
                            contraindications: "{properties_dict.get("contraindications", "")}",
                            adverse_effects: "{properties_dict.get("adverse_effects", "")}"
                        }}]->(node1);
                    """
                elif edge in ["INCREASED_ABSORPTION", "INCREASED_EFFECT"]:
                    interaction_type = "Increased Effect"
                    cypher_query = f"""
                        OPTIONAL MATCH (node1:Drug {{Drug_name: "{node1}"}})
                        OPTIONAL MATCH (node2:Drug {{Drug_name: "{node2}"}})
                        WHERE node1 IS NOT NULL AND node2 IS NOT NULL
                        MERGE (node1)-[:HAS_INTERACTION_NLP {{
                            type: "{interaction_type}",
                            severity: "{str(properties_dict.get("severity", ""))}",
                            gender: "{properties_dict.get("gender", "")}",
                            pregnancy: "{properties_dict.get("pregnancy", "")}",
                            contraindications: "{properties_dict.get("contraindications", "")}",
                            adverse_effects: "{properties_dict.get("adverse_effects", "")}"
                        }}]->(node2)
                        MERGE (node2)-[:HAS_INTERACTION_NLP {{
                            type: "{interaction_type}",
                            severity: "{str(properties_dict.get("severity", ""))}",
                            gender: "{properties_dict.get("gender", "")}",
                            pregnancy: "{properties_dict.get("pregnancy", "")}",
                            contraindications: "{properties_dict.get("contraindications", "")}",
                            adverse_effects: "{properties_dict.get("adverse_effects", "")}"
                        }}]->(node1);
                    """
                # Ejecuta la consulta Cypher
                session.run(cypher_query)


In [21]:
uri = "bolt://localhost:7687" 
username = "neo4j"
password = "TFMLAURA"
create_interaction_relationships("data_output/cureus2/graph.csv",uri,username,password)


In [13]:
from neo4j import GraphDatabase
import gradio as gr


def buscar_interacciones(Drug1, Drug2):
    # Input drugs are converted to the format first letter upper case and the rest are lower case
    Drug1 = Drug1.lower().capitalize()
    Drug2 = Drug2.lower().capitalize()
    
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        # Open connection with Neo4j
        with driver.session() as session:
            # Cypher query in Neo4j
            query = (
                "MATCH (d1:Drug {Drug_name: $Drug1})-[:INTERACTS_WITH]-(d2:Drug {Drug_name: $Drug2}) "
                "RETURN d1.Drug_name, d2.Drug_name"
            )
            result = session.run(query, Drug1=Drug1, Drug2=Drug2)
            # List of interactions founded
            interactions = [record.values() for record in result]

            query2 = (
                "MATCH (d1:Drug {Drug_name: $Drug1})-[:HAS_INTERACTION_NLP]-(d2:Drug {Drug_name: $Drug2}) "
                "RETURN d1.Drug_name, d2.Drug_name"
            )
            result2 = session.run(query2, Drug1=Drug1, Drug2=Drug2)
            # List of interactions found in articles of PubMed
            interactions2 = [record.values() for record in result2]
# Returned result
    if interactions and not interactions2:
        return f"Interactions found: {interactions}"

    elif not interactions and interactions2:
        return f"Interactions found by processing articles from PubMed: {interactions2}"

    elif interactions and interactions2:
        return f"Interactions found: {interactions}\nInteractions found by processing articles from PubMed: {interactions2}"

    else:
        return "No interactions founded."

/Users/lauraolivaresbenitez/opt/anaconda3/envs/knowledge-graph/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
def all_ddi(Drug):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        # The input drug is converted to the format first letter upper case and the rest are lower case
        Drug = Drug.lower().capitalize()
        #Open conexion with Neo4j
        with driver.session() as session:
            # Cypher query in Neo4j
            query = (
               "MATCH (d1:Drug {Drug_name: $Drug})-[:INTERACTS_WITH]-(d2:Drug)"
               "RETURN d2.Drug_name"
            )
            result = session.run(query, Drug=Drug)
            # List of intercations founded
            drug_interactions = [record["d2.Drug_name"] for record in result]
            query2 = (
               "MATCH (d1:Drug {Drug_name: $Drug})-[:HAS_INTERACTION_NLP]-(d2:Drug)"
               "RETURN d2.Drug_name"
            )
            result2 = session.run(query, Drug=Drug)
            # List of intercations founded
            drug_interactions2 = [record["d2.Drug_name"] for record in result2]

    # Returned result
    if drug_interactions and not drug_interactions2:
        return f"{Drug} interacts with: {drug_interactions}"
    elif not drug_interactions and drug_interactions2:
        return f"Found in processed articles from PubMed that {Drug} interacts with: {drug_interactions2}"
    elif drug_interactions and drug_interactions2:
        return f"{Drug} interacts with: {drug_interactions} \nFound in processed articles from PubMed that {Drug} interacts with: {drug_interactions2}"
    else:
        return f"No drug interactions founded with {Drug}."

In [72]:
with gr.Blocks() as demo:
    gr.Markdown("## <center>Check or find drug interactions</center>")
    with gr.Tabs():
        with gr.Tab("Drug Interactions Checker"):
            interaction_input1 = gr.Textbox(label="Drug 1",placeholder="Insert a drug here")
            interaction_input2 = gr.Textbox(label="Drug 2",placeholder="Insert a drug here")
            interaction_output = gr.Textbox(label="Interactions Result")
            interaction_button = gr.Button("Check Drug Interactions")

            interaction_button.click(buscar_interacciones, inputs=[interaction_input1, interaction_input2], outputs=interaction_output)

        with gr.Tab("Search Interacting Medications"):
            Drug = gr.Textbox(label="Drug",placeholder="Insert a drug here")
            medication_output = gr.Textbox(label="Interacting Medications Result")
            medication_button = gr.Button("Search Interacting Medications")

            medication_button.click(all_ddi, inputs=Drug, outputs=medication_output)

if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://457d024e8e1cef8874.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
